<a href="https://colab.research.google.com/github/mzignis/hands_on_machine_learning/blob/master/%20artificial_neural_networks/introduction_to_ann_with_keras/hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from scipy.stats import reciprocal

import tensorflow as tf
from tensorflow import keras

In [2]:
housing = fetch_california_housing()

In [0]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full)
X_new = X_test[:3]

In [0]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=1e-3, input_shape=[8]):
    # init model
    model = keras.models.Sequential()
    options = {'input_shape': input_shape}

    # hidden layers
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu', **options))
        options = {}
    
    # output layer
    model.add(keras.layers.Dense(1, **options))

    # compile model
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)

    return model


In [0]:
# get keras wrapper - it let use it as "normal" sklearn model

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [6]:
keras_reg.fit(X_train, y_train,
              epochs=100,
              validation_data=(X_val, y_val),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 5328176439885824.0000 - val_loss: 6953099067392.0000
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 3672911642624.0000 - val_loss: 1625369346048.0000
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 858586152960.0000 - val_loss: 379949252608.0000
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 200704524288.0000 - val_loss: 88817483776.0000
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 46917029888.0000 - val_loss: 20762103808.0000
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 10967393280.0000 - val_loss: 4853387264.0000
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 2563756544.0000 - val_loss: 1134537728.0000
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 599308544.0000 - val_loss: 265211376.0000
Epoch 9/100
363/363 [====

In [8]:
param_distribs = {
    "n_hidden": [0, 1, 2],
    "n_neurons": np.arange(30, 50),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, 
                                   param_distributions=param_distribs,
                                   n_iter=10,
                                   cv=2)

rnd_search_cv.fit(X_train, y_train,
                  epochs=100,
                  validation_data=(X_val, y_val),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
182/182 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 2/100
182/182 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
182/182 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
182/182 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
182/182 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 6/100
182/182 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
182/182 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 8/100
182/182 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/100
182/182 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
182/182 [==============================] - 0s 1ms/step - loss: nan
Epoch 1/100
182/182 [==============================] - 1s 3ms/ste

RuntimeError: ignored

In [9]:
rnd_search_cv.best_params_

{'learning_rate': 0.0003541689630416509, 'n_hidden': 1, 'n_neurons': 35}

In [10]:
rnd_search_cv.best_score_

-4.749833047389984